In [4]:
import requests
import bs4
import re
import pandas as pd

In [5]:
index_url = 'http://www.presidency.ucsb.edu/nomination.php'
r = requests.get(index_url)
contents = r.text

ConnectionError: HTTPConnectionPool(host='www.presidency.ucsb.edu', port=80): Max retries exceeded with url: /nomination.php (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x116e704e0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [ ]:
doc = bs4.BeautifulSoup(contents, "html")

In [ ]:
data_rows = doc.find_all('table')
democrat = None
repub = None
for i,table in enumerate(data_rows):
    obama = 'Obama' in str(table)
    bush = 'Bush' in str(table)
    if obama and not bush:
        democrat = i
    if bush and not obama:
        repub = i

        
empty_p = re.compile('[\n\r\s]+')    
def extract_party_data(party_rows):
    party_header = ['date','speech_link','nominee','location','state_win']
    party_data = []
    for j,row in enumerate(party_rows):
        values = row.find_all('td')
        if j != 0:
            date,nominee,blank,location,state_win = values
            date = re.sub(empty_p,' ', date.text)
            speech_link = ""
            if nominee.find('a'):
                speech_link = nominee.find('a')['href']
            nominee = re.sub(empty_p,' ', nominee.text)
            location = re.sub(empty_p,' ', location.text)
            state_win = re.sub(empty_p,' ', state_win.text)
            party_data.append([date,speech_link,nominee,location,state_win])
            
    party_df = pd.DataFrame(party_data, columns=party_header)
    return party_df

democrat_rows = data_rows[democrat].find_all('tr')
democrat_df = extract_party_data(democrat_rows).head()

repub_rows = data_rows[repub].find_all('tr')
repub_df = extract_party_data(repub_rows).head()

In [ ]:
help(re.split)

In [ ]:
class SpeechExtractor():
    speaker_p = re.compile('(<p>)*<i>(.+)</i>([^<]*)(</p>)*')
    def __init__(self, url):
        self.link = url
        r = requests.get(self.link)
        self.contents = r.text
        self.doc = bs4.BeautifulSoup(self.contents, "html")
        self.speech = {}
        self.parse_speech()
        
    def __update_speech__(self, speaker, lines):
        self.speech[speaker] = self.speech.get(speaker, []) + lines
        
    def parse_speech(self):
        speech = self.doc.find('span', attrs={'class':'displaytext'})#.find_all('p')
        #lines = [x for x in speech.children]
        current_speaker = None
        current_contents = []

        for line in speech.children:
            if line:
                speaker_match = re.match(SpeechExtractor.speaker_p, str(line).strip())
                if isinstance(line,bs4.element.NavigableString):
                    current_contents.append(line.strip())
                elif speaker_match:
                    p_tag, speaker, text, p_close = speaker_match.groups()
                    if 'U.S.A' in str(line):
                        print(speaker_match.groups())
                    self.__update_speech__(current_speaker, current_contents)
                    current_speaker = speaker.strip()
                    current_contents = []
                    if text:
                        current_contents.append(text.strip())
                elif str(line).startswith('<p>'):
                    current_contents.append(line.text.rstrip('</p>').strip())
                    
                else:
                    print('UNCLASSIFIED', type(line),line)    
                '''
                elif isinstance(line,bs4.Tag):
                    speech_part = line.find('i')
                    if speech_part:
                        speaker = speech_part.text
                        if '.' in speaker:#'Laughter' not in speaker and 'Applause' not in speaker:
                            self.__update_speech__(current_speaker, current_contents)
                            print(line)
                            current_speaker, current_contents = line.split('</i>')
                            current_contents = [speech_part.text.strip()]
                        else:
                            current_contents.append(line.text.strip())
                    else:
                        print(line)
                        speaker_match = re.match(speaker_p, str(line))
                        if speaker_match:
                            speaker, text = speaker_match.groups()
                            if current_contents:
                                self.__update_speech__(speaker, current_contents)
                            current_speaker = speaker
                            if text:
                                current_contents = [text]
                            else:
                                current_contents = []
                        else:
                            current_contents.append(line.text.strip())

                else:
                    print('UNCLASSIFIED', type(line),line)
                '''

            
        
new_speech = SpeechExtractor('http://www.presidency.ucsb.edu/ws/index.php?pid=101968')
print(new_speech.speech.keys())

In [ ]:
sample = new_speech.doc.find('span', attrs={'class':'displaytext'}).contents.copy()

In [ ]:
parsed_data = ''
for line in sample:
    line = str(line).strip()
    if line:
        if line.startswith('<p>'):
            line = line[3:-4]
        parsed_data += line.strip()
            


In [ ]:
parsed_data

In [ ]:
p_most = re.compile('<i>([^\.]+?)\.</i>(.*?)(\[*<i>[^\.]+?\.</i>\]*)')

parts = re.findall(p_most,parsed_data)

speaker_d = {}
for part in parts:
    speaker, words, end = part
    speaker_d[speaker] = speaker_d.get(speaker, []) + [words]
parts

In [ ]:
parts

In [ ]:
dir(re)

In [ ]:
help(re.search)

In [ ]:
s = 'Hello my name is'
p = re.compile('Hello')

m = re.search(p,s)

In [ ]:
dir(m)

In [ ]:
m.span()

In [ ]:
while parsed_data:

In [ ]:
start_index = 0
speaker_p = re.compile('<i>([^\.])+\.</i>')
while start_index < len(parsed_data):
    m = re.search(speaker_p, parsed_data)
    if m:
        print(m.group(0))
        start_index = m.span()[1]
    else:
        start_index = len(parsed_data)